# カタカナ15文字を自動識別するモデルの構築
# 課題投稿用Notebook
* これは課題投稿用ファイルです。
* このsubmit_katakana.ipynbファイルを編集し、学習済みモデルなどの必要ファイルを同じフォルダにおき、そのフォルダをzipしたものを投稿してください。

In [9]:
from tqdm import tqdm
from time import sleep
pbar = tqdm(["a", "b", "c", "d"])
v = 0
for char in pbar:
    v += 1
    sleep(0.25)
    pbar.set_description(f'v: {v}')

v: 4: 100%|██████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.96it/s]


In [1]:
import numpy as np
import glob
import os,sys
import util
import pickle
# %load_ext autoreload
# %autoreload 2
from common.model import Conv2DNN

def makedataset():
    """
    データセットをつくる関数です。
    自由に編集してください。
    """
    
    # 次の行は変更しないこと
    test_data = util.loaddata()
    
    # 以下は自由に編集しても構いません
    # 必要な前処理をここに記述してください  
    
    # 正規化
    test_data = test_data / test_data.max()
    test_data = test_data.astype('float32')

    # 配列形式変更
    # test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    return test_data


def func_predict(test_data, test_label):
    """
    予測する関数
    data : 画像データ
    return loss, accuracy
    引数とreturn以外は、自由に編集してください    
    """
    # 以下を自由に編集してください
    # with open("katakana_model.pickle", "rb") as f:
    #     model = pickle.load(f)
    kwargs = {
        'conv_param': {'filter_num': 64, 'filter_size': 3, 'pad': 0, 'stride': 1},
        'pool_param': {'pool_size': 2, 'pad': 0, 'stride': 2},
        'hidden_size': 64,
        'learning_rate': 1e-3, 'epsilon': 1e-8, # Compile HP
        'batch_size': 32, 'epochs': 100, 'patience': 4, # fit HP
    }
    model = Conv2DNN(
        input_dim=test_data.shape[1:],
        conv_param=kwargs['conv_param'],
        pool_param=kwargs['pool_param'],
        hidden_size=kwargs['hidden_size'],
        output_size=15,
        weight_init_std=0.01,
    )
    with open("katakana_model_params.pickle", "rb") as f:
        params = pickle.load(f)
        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2',]):
            model.layers[key].W = params['W' + str(i+1)]
            model.layers[key].b = params['b' + str(i+1)]
            # model.params[key].W = params['W' + str(i+1)]
            # model.params[key].b = params['b' + str(i+1)]
    
    X_test = test_data
    y_test = test_label
    
    # calculate loss, accuracy
    accuracy = model.accuracy(X_test, y_test)
    loss  = model.loss(X_test, y_test)
    
    return loss, accuracy # 編集不可


def main():
    """
    編集しないでください。
    """
    # テスト用データをつくる
    test_data = makedataset()

    # 予測し精度を算出する
    util.accuracy(func_predict, test_data)
    
    return


if __name__=="__main__":
    main()

0
0.21008313655613392 0.966
1
0.2154216199277033 0.96
2
0.09880022225754731 0.98
3
0.18193948135453136 0.97
4
0.196523216565544 0.956
5
0.2421183234524076 0.958
Test loss: 0.1908143333523112
Test accuracy: 0.965
